<a href="https://colab.research.google.com/github/tselane2110/Brain-Tumor-Detection-And-Classification/blob/main/implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tselane2110/SSCLNet-Implementation

Cloning into 'SSCLNet-Implementation'...
remote: Enumerating objects: 307, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (132/132), done.
Receiving objects: 100% (307/307), 589.57 KiB | 7.86 MiB/s, done.
remote: Total 307 (delta 95), reused 74 (delta 32), pack-reused 143 (from 1)
Resolving deltas: 100% (170/170), done.


In [ ]:
%cd /content/SSCLNet-Implementation

/content/SSCLNet-Implementation


## 1. Data Loading & Preprocessing

In [ ]:
import dataset

In [ ]:
dataset.load_data("https://drive.google.com/file/d/1Ke4oXJiZiUnomAwUfjjlmxzeI6HK1Zz4/view?usp=drive_link")

Downloading...
From (original): https://drive.google.com/uc?id=1Ke4oXJiZiUnomAwUfjjlmxzeI6HK1Zz4
From (redirected): https://drive.google.com/uc?id=1Ke4oXJiZiUnomAwUfjjlmxzeI6HK1Zz4&confirm=t&uuid=15b8be63-fa48-4326-ba61-93b106a6d06c
To: /content/dataset.zip
100%|██████████| 429M/429M [00:01<00:00, 287MB/s]


Extracting dataset...
Dataset ready at: /content/


'/content/'

In [ ]:
# moving dataset folder
!mv /content/content/Dataset-Brain-MRI /content

In [ ]:
# deleting the extra "content" folder
!rm -r /content/content

In [ ]:
# deleting the "duplicates_backup" folder from the "Dataset-Brain-MRI" folder, since we wont be using it anyways
!rm -r /content/Dataset-Brain-MRI/duplicates_backup

In [ ]:
dataset.split_data_disjoint_pretrain("/content/Dataset-Brain-MRI", "/content/splitted-dataset")

Starting DISJOINT dataset splitting (70% pretrain, 20% train, 10% test)...
Input directory: /content/Dataset-Brain-MRI
Output directory: /content/splitted-dataset
Created: /content/splitted-dataset/pretrain
Created: /content/splitted-dataset/train/2-class/yes
Created: /content/splitted-dataset/train/2-class/no
Created: /content/splitted-dataset/train/4-class/glioma_tumor
Created: /content/splitted-dataset/train/4-class/meningioma_tumor
Created: /content/splitted-dataset/train/4-class/no_tumor
Created: /content/splitted-dataset/train/4-class/pituitary_tumor
Created: /content/splitted-dataset/test/2-class/yes
Created: /content/splitted-dataset/test/2-class/no
Created: /content/splitted-dataset/test/4-class/glioma_tumor
Created: /content/splitted-dataset/test/4-class/meningioma_tumor
Created: /content/splitted-dataset/test/4-class/no_tumor
Created: /content/splitted-dataset/test/4-class/pituitary_tumor

Total images collected: 11073

Copying files...

=== SPLITTING COMPLETED ===
Pre-train

In [ ]:
dataset.preprocess_split_data("/content/splitted-dataset", "/content/preprocessed-splitted-data")

Preprocessing split dataset with new structure...
Input path: /content/splitted-dataset
Output path: /content/preprocessed-splitted-data

Processing pretrain folder...
  Pretrain: 7666 images processed

Processing train folder...
  Train/2-class/yes: 617 images processed
  Train/2-class/no: 582 images processed
  Train/4-class/glioma_tumor: 429 images processed
  Train/4-class/meningioma_tumor: 244 images processed
  Train/4-class/no_tumor: 143 images processed
  Train/4-class/pituitary_tumor: 199 images processed

Processing test folder...
  Test/2-class/yes: 299 images processed
  Test/2-class/no: 317 images processed
  Test/4-class/glioma_tumor: 226 images processed
  Test/4-class/meningioma_tumor: 109 images processed
  Test/4-class/no_tumor: 67 images processed
  Test/4-class/pituitary_tumor: 90 images processed

=== PREPROCESSING COMPLETED ===
Total images successfully processed: 10988
Total errors: 0
Preprocessed dataset saved to: /content/preprocessed-splitted-data

Final struc

In [ ]:
!zip -r "/content/preprocessed_splitted_data.zip" "/content/preprocessed-splitted-data"

Streaming output truncated to the last 5000 lines.
  adding: content/preprocessed-splitted-data/pretrain/no@4555.jpg (deflated 5%)
  adding: content/preprocessed-splitted-data/pretrain/yes@2100.jpg (deflated 2%)
  adding: content/preprocessed-splitted-data/pretrain/m3 (190).jpg (deflated 4%)
  adding: content/preprocessed-splitted-data/pretrain/m (57).jpg (deflated 3%)
  adding: content/preprocessed-splitted-data/pretrain/Tr-gl_0468.jpg (deflated 3%)
  adding: content/preprocessed-splitted-data/pretrain/Tr-no_0611.jpg (deflated 3%)
  adding: content/preprocessed-splitted-data/pretrain/gg (212).jpg (deflated 3%)
  adding: content/preprocessed-splitted-data/pretrain/no@2228.jpg (deflated 3%)
  adding: content/preprocessed-splitted-data/pretrain/yes@1857.jpg (deflated 4%)
  adding: content/preprocessed-splitted-data/pretrain/no@4664.jpg (deflated 2%)
  adding: content/preprocessed-splitted-data/pretrain/no@4922.jpg (deflated 3%)
  adding: content/preprocessed-splitted-data/pretrain/yes@49

* Downloaded the `preprocessed_splitted_data.zip` file manually and uploaded it to the gdrive folder.

## 2. Training Phase 1 - Contrastive Learning

In [1]:
!git clone https://github.com/tselane2110/SSCLNet-Implementation

Cloning into 'SSCLNet-Implementation'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 316 (delta 14), reused 5 (delta 4), pack-reused 297 (from 1)
Receiving objects: 100% (316/316), 545.57 KiB | 2.09 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [3]:
# loading the preprocessed-splitted dataset
!gdown --fuzzy "https://drive.google.com/file/d/16qJoMaEM4VRIVqS7-iKwGfga-wq5x_lk/view?usp=drive_link"
!unzip -q /content/preprocessed_splitted_data.zip

Downloading...
From (original): https://drive.google.com/uc?id=16qJoMaEM4VRIVqS7-iKwGfga-wq5x_lk
From (redirected): https://drive.google.com/uc?id=16qJoMaEM4VRIVqS7-iKwGfga-wq5x_lk&confirm=t&uuid=2f213aff-f72a-432b-8cc4-4dddad9890f2
To: /content/preprocessed_splitted_data.zip
100% 153M/153M [00:01<00:00, 130MB/s]


In [4]:
# moving folders
!mv /content/content/preprocessed-splitted-data /content/

In [5]:
# deleting the extra "content" folder
!rm -r /content/content

In [ ]:
# update file paths for pretrain and train data in the config.py file

In [7]:
!python "/content/SSCLNet-Implementation/train_contrastive.py"

2025-11-29 06:55:22.053397: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764399322.072513    2450 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764399322.078324    2450 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764399322.093058    2450 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764399322.093082    2450 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764399322.093086    2450 computation_placer.cc:177] computation placer alr

In [ ]:
# downloading the output
# !zip -r contrastive_training_output.zip /content/contrastive_training_output

## 3. Training Phase 2 - Supervised Learning (4-class dataset)

In [ ]:
# !git clone https://github.com/tselane2110/SSCLNet-Implementation

In [ ]:
# loading the dataset
# !gdown --fuzzy "https://drive.google.com/file/d/16-v8HBPN_Nnj8JnQSDRSnfq5C1YqD-8y/view?usp=drive_link"
# !unzip -q /content/Preprocessed-splitted-data.zip

In [8]:
!python "/content/SSCLNet-Implementation/train_supervised.py"

2025-11-29 11:10:14.349743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764414614.381184   67567 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764414614.390953   67567 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764414614.414505   67567 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764414614.414555   67567 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764414614.414565   67567 computation_placer.cc:177] computation placer alr

## 4. Model Evaluation (4-class dataset)

In [9]:
# testing / model evaluation:
!python "/content/SSCLNet-Implementation/eval.py"

2025-11-29 11:11:28.730832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764414688.761053   67955 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764414688.770672   67955 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764414688.793655   67955 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764414688.793691   67955 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764414688.793698   67955 computation_placer.cc:177] computation placer alr

In [10]:
!zip -r /content/plots_4Class.zip /content/plots

  adding: content/plots/ (stored 0%)
  adding: content/plots/contrastive_loss.png (deflated 24%)
  adding: content/plots/roc_curve.png (deflated 20%)
  adding: content/plots/class_distribution.png (deflated 28%)
  adding: content/plots/confusion_matrix.png (deflated 26%)
  adding: content/plots/supervised_training_history.png (deflated 28%)


In [11]:
!zip -r /content/logs_4Class.zip /content/logs

  adding: content/logs/ (stored 0%)
  adding: content/logs/supervised_finetuning.log (deflated 91%)
  adding: content/logs/contrastive_pretraining.log (deflated 89%)
  adding: content/logs/model_evaluation.log (deflated 79%)


## 5. Training Phase 2 - Supervised Learning (2-class dataset)

**Note:** Update the path to `TRAIN_DATA_PATH`, `TEST_DATA_PATH` and `NUM_CLASSES` in `config.py` to the 2-class dataset.

In [15]:
!python "/content/SSCLNet-Implementation/train_supervised.py"

2025-11-29 11:30:19.337051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764415819.356851   72774 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764415819.362907   72774 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764415819.377926   72774 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764415819.377952   72774 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764415819.377956   72774 computation_placer.cc:177] computation placer alr

## 6. Model Evaluation (2-class dataset)

In [19]:
# testing / model evaluation:
!python "/content/SSCLNet-Implementation/eval.py"

2025-11-29 11:45:20.714711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764416720.735372   76692 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764416720.741418   76692 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764416720.756860   76692 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764416720.756885   76692 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764416720.756888   76692 computation_placer.cc:177] computation placer alr

In [20]:
!zip -r /content/plots_2Class.zip /content/plots
!zip -r /content/results_2Class.zip /content/results
!zip -r /content/logs_2Class.zip /content/logs

  adding: content/plots/ (stored 0%)
  adding: content/plots/contrastive_loss.png (deflated 24%)
  adding: content/plots/roc_curve.png (deflated 24%)
  adding: content/plots/class_distribution.png (deflated 41%)
  adding: content/plots/confusion_matrix.png (deflated 25%)
  adding: content/plots/supervised_training_history.png (deflated 25%)
  adding: content/results/ (stored 0%)
  adding: content/results/test_results.txt (deflated 40%)
  adding: content/results/final_evaluation_results.json (deflated 53%)
  adding: content/logs/ (stored 0%)
  adding: content/logs/supervised_finetuning.log (deflated 91%)
  adding: content/logs/contrastive_pretraining.log (deflated 89%)
  adding: content/logs/model_evaluation.log (deflated 88%)
